In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,47119
2,Huelva,Confirmados PDIA 14 días,663
3,Huelva,Tasa PDIA 14 días,"129,1969522770232"
4,Huelva,Confirmados PDIA 7 días,342
5,Huelva,Tasa PDIA 7 dias,"66,64458171755949"
6,Huelva,Total Confirmados,47323
7,Huelva,Curados,45666
8,Huelva,Fallecidos,423


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  47119.0


/tmp/ipykernel_74747/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13037.0


/tmp/ipykernel_74747/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_74747/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_74747/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_74747/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 999 personas en los últimos 7 días 

Un positivo PCR cada 543 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,47119.0,342.0,663.0,513170.0,66.644582,129.196952,44.0
Huelva-Costa,27863.0,207.0,396.0,289548.0,71.490737,136.764889,30.0
Huelva (capital),13037.0,144.0,265.0,143837.0,100.113323,184.236323,23.0
Condado-Campiña,14683.0,120.0,235.0,156231.0,76.809340,150.418291,14.0
Paterna del Campo,226.0,11.0,14.0,3457.0,318.194967,404.975412,4.0
Cartaya,2224.0,30.0,67.0,20083.0,149.380073,333.615496,3.0
Aljaraque,1670.0,14.0,21.0,21474.0,65.195120,97.792680,2.0
Bollullos Par del Condado,1341.0,11.0,29.0,14387.0,76.457913,201.570863,2.0
Moguer,2089.0,28.0,47.0,21867.0,128.046829,214.935748,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),10.0,4.0,5.0,243.0,1646.090535,2057.613169,0.0
Palos de la Frontera,1151.0,25.0,53.0,11742.0,212.910918,451.371146,1.0
Paterna del Campo,226.0,11.0,14.0,3457.0,318.194967,404.975412,4.0
Cartaya,2224.0,30.0,67.0,20083.0,149.380073,333.615496,3.0
Campofrío,61.0,1.0,2.0,713.0,140.252454,280.504909,NaN
San Bartolomé de la Torre,398.0,3.0,10.0,3761.0,79.766020,265.886732,0.0
Palma del Condado (La),1467.0,16.0,27.0,10801.0,148.134432,249.976854,1.0
Moguer,2089.0,28.0,47.0,21867.0,128.046829,214.935748,2.0
Alosno,240.0,6.0,8.0,3933.0,152.555301,203.407068,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Isla Cristina,3194.0,3.0,17.0,21393.0,14.023279,79.465246,0.0,0.176471
Ayamonte,2117.0,1.0,4.0,21104.0,4.738438,18.953753,1.0,0.250000
San Bartolomé de la Torre,398.0,3.0,10.0,3761.0,79.766020,265.886732,0.0,0.300000
Trigueros,551.0,2.0,6.0,7862.0,25.438820,76.316459,0.0,0.333333
Bollullos Par del Condado,1341.0,11.0,29.0,14387.0,76.457913,201.570863,2.0,0.379310
Gibraleón,967.0,6.0,15.0,12737.0,47.106854,117.767135,0.0,0.400000
Lepe,3058.0,2.0,5.0,27880.0,7.173601,17.934003,1.0,0.400000
Cartaya,2224.0,30.0,67.0,20083.0,149.380073,333.615496,3.0,0.447761
Palos de la Frontera,1151.0,25.0,53.0,11742.0,212.910918,451.371146,1.0,0.471698
